### Import dependencies

In [13]:
import json
import sqlite3
import os
from datetime import datetime

from data_api import youtube
from helpers import dict_search

### Import Google Data API Key and Initiate Youtube API Class Instance
1. Create Youtube Data API as per the instructions at: https://developers.google.com/youtube/v3/getting-started \

2. Set environment variable containing the API key by executing: `export YT_API_KEY=[insert private key here]`

In [16]:
try:
    DEVELOPER_KEY = os.environ['YT_API_KEY']
except KeyError:
    print("Unable to find API key environement variable!")

Unable to find API key environement variable!


In [14]:
yt = youtube(DEVELOPER_KEY)

KeyError: 'YT_API_KEY'

In [3]:
dt = datetime.now()

raw_categories = yt.VideoCategories('GB')
categories = dict_search(raw_categories, ["id", "title", "assignable"])
    
for cat in categories:
    cat['region'] = 'GB'
    cat['time_updated'] = dt

try: 
    with sqlite3.connect('yt_sentiment.db') as conn:
        sql = "INSERT OR REPLACE INTO \
            categories(category_id, title, assignable, region, time_updated) \
            VALUES(:id, :title, :assignable, :region, :time_updated)"
        conn.executemany(sql, categories)
        print("Categories table updated successfully!")
        
except sqlite3.Error as error:
    print("Error while creating a sqlite table", error)

Categories table updated successfully!
